In [1]:
!pip install internetarchive requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.9/105.9 kB 3.3 MB/s eta 0:00:00


In [2]:
import os
import requests
from internetarchive import search_items, get_item, download

def search_internet_archive(query, max_results=5):
    """
    Search IA for items that mention `query` in title OR subject OR description.
    Returns up to max_results dicts with 'identifier' and 'title'.
    """
    # broadened JQL to search multiple metadata fields
    jql = (
        f'title:({query}) '
        f'OR subject:({query}) '
        f'OR description:({query})'
    )
    results = []
    for count, item in enumerate(search_items(jql), start=1):
        results.append({
            'identifier': item['identifier'],
            'title': item.get('title', '(no title)')
        })
        if count >= max_results:
            break
    return results

def download_from_internet_archive(
    identifier,
    download_dir='downloads',
    max_downloads=10,
    extensions=('.pdf', '.lcpdf', '.txt', '.djvu', '.epub')
):
    """
    Download up to max_downloads files for `identifier` whose names end
    with one of extensions, skipping only those explicitly marked restricted.
    """
    item = get_item(identifier)
    os.makedirs(os.path.join(download_dir, identifier), exist_ok=True)

    to_fetch = []
    for f in item.files:
        name = f.get('name', '')
        if any(name.lower().endswith(ext) for ext in extensions):
            if f.get('restricted') == '1':
                print(f"Skipping restricted: {name}")
                continue
            to_fetch.append(name)
            if len(to_fetch) >= max_downloads:
                break

    if not to_fetch:
        print(f"No public files with extensions {extensions} for '{identifier}'")
        return

    try:
        download(
            identifier,
            files=to_fetch,
            destdir=os.path.join(download_dir, identifier),
            verbose=True
        )
    except Exception as e:
        print(f"Error downloading from {identifier}: {e}")

def download_gutenberg_text(book_id, save_dir='downloads/gutenberg'):
    """
    Download the plain-text version of a Gutenberg book by its ID.
    """
    os.makedirs(save_dir, exist_ok=True)
    for variant in (f'{book_id}-0.txt', f'{book_id}.txt'):
        url = f'https://www.gutenberg.org/files/{book_id}/{variant}'
        try:
            r = requests.get(url)
            r.raise_for_status()
            path = os.path.join(save_dir, f'gutenberg_{book_id}.txt')
            with open(path, 'w', encoding='utf-8') as out:
                out.write(r.text)
            print(f"Downloaded Gutenberg book {book_id} as {variant}")
            return
        except requests.HTTPError:
            continue
    print(f"No text file found for Gutenberg ID {book_id}")

if __name__ == '__main__':
    # 1) search & download IA items for "Amazon archaeology"
    items = search_internet_archive('Amazon archaeology', max_results=100)
    for itm in items:
        print(f"Found “{itm['title']}” (ID: {itm['identifier']})")
        download_from_internet_archive(
            itm['identifier'],
            max_downloads=10
        )

    # 2) Gutenberg examples (optional)
    for gid in (12472, 22752):
        download_gutenberg_text(gid)

Found “(no title)” (ID: 2010-mattievich-enrico-journey-to-the-mythological-inferno-americas-discovery-by)


2010-mattievich-enrico-journey-to-the-mythological-inferno-americas-discovery-by:
 downloading 1995 Enrico Mattievich ~ Tαξίδι στη μυθολογική κόλαση - Η ανακάλυψη της Αμερικής από τους Έλληνες  #1#.pdf: 100%|██████████| 9.91M/9.91M [00:00<00:00, 13.4MiB/s]
 downloading 1995 Enrico Mattievich ~ Tαξίδι στη μυθολογική κόλαση - Η ανακάλυψη της Αμερικής από τους Έλληνες  #1#_djvu.txt: 100%|██████████| 480k/480k [00:00<00:00, 1.58MiB/s]
 downloading 2010 Mattievich, Enrico ~ Journey to the Mythological Inferno_ America's Discovery by the Ancient Greeks [Rogem Press] _.pdf: 100%|██████████| 34.2M/34.2M [00:01<00:00, 23.0MiB/s]
 downloading 2010 Mattievich, Enrico ~ Journey to the Mythological Inferno_ America's Discovery by the Ancient Greeks [Rogem Press] __djvu.txt: 100%|██████████| 424k/424k [00:00<00:00, 1.41MiB/s]


Found “(no title)” (ID: 3CreepyTv)
No public files with extensions ('.pdf', '.lcpdf', '.txt', '.djvu', '.epub') for '3CreepyTv'
Found “(no title)” (ID: 4lv08yhcls19uobhwcsegehdjg8x2ptbysyaev8x)
No public files with extensions ('.pdf', '.lcpdf', '.txt', '.djvu', '.epub') for '4lv08yhcls19uobhwcsegehdjg8x2ptbysyaev8x'
Found “(no title)” (ID: ABudABowlShowHD420PodcastHistoryOfCannabisInTheAncientWorldBCPartTwo)
No public files with extensions ('.pdf', '.lcpdf', '.txt', '.djvu', '.epub') for 'ABudABowlShowHD420PodcastHistoryOfCannabisInTheAncientWorldBCPartTwo'
Found “(no title)” (ID: AlfredWebreAudio)
No public files with extensions ('.pdf', '.lcpdf', '.txt', '.djvu', '.epub') for 'AlfredWebreAudio'
Found “(no title)” (ID: AncientSuppressedDiscoveriesWithJonathanGray)
No public files with extensions ('.pdf', '.lcpdf', '.txt', '.djvu', '.epub') for 'AncientSuppressedDiscoveriesWithJonathanGray'
Found “(no title)” (ID: AssemblyLinesPodcast83)
No public files with extensions ('.pdf', '.lcpdf

ERIC_ED371971:
 downloading ERIC_ED371971.djvu: 100%|██████████| 3.67M/3.67M [00:00<00:00, 5.45MiB/s]
 downloading ERIC_ED371971.pdf: 100%|██████████| 1.35M/1.35M [00:00<00:00, 3.05MiB/s]
 downloading ERIC_ED371971_djvu.txt: 124kiB [00:00, 1.59MiB/s]


Found “(no title)” (ID: KIDE_91_3_FM_20170217_170000)
No public files with extensions ('.pdf', '.lcpdf', '.txt', '.djvu', '.epub') for 'KIDE_91_3_FM_20170217_170000'
Found “(no title)” (ID: KIDE_91_3_FM_20170218_020000)
No public files with extensions ('.pdf', '.lcpdf', '.txt', '.djvu', '.epub') for 'KIDE_91_3_FM_20170218_020000'
Found “(no title)” (ID: KIDE_91_3_FM_20170224_170000)
No public files with extensions ('.pdf', '.lcpdf', '.txt', '.djvu', '.epub') for 'KIDE_91_3_FM_20170224_170000'
Found “(no title)” (ID: KPFA_94_1_FM_20180312_010000)
No public files with extensions ('.pdf', '.lcpdf', '.txt', '.djvu', '.epub') for 'KPFA_94_1_FM_20180312_010000'
Found “(no title)” (ID: KSKA_91_1_FM_20170218_050000)
No public files with extensions ('.pdf', '.lcpdf', '.txt', '.djvu', '.epub') for 'KSKA_91_1_FM_20170218_050000'
Found “(no title)” (ID: ManlyPhall)


...

Error downloading from postcolonialamaz0000penr_e8i4: 401 Client Error: Unauthorized for url: https://dn720006.ca.archive.org/0/items/postcolonialamaz0000penr_e8i4/postcolonialamaz0000penr_e8i4.pdf
Found “(no title)” (ID: retncrv2006vtarchaeologyjpetersentribute)
No public files with extensions ('.pdf', '.lcpdf', '.txt', '.djvu', '.epub') for 'retncrv2006vtarchaeologyjpetersentribute'
Found “(no title)” (ID: rwugbiupl1umttkibxzx4vjh4jd0c8nszblewbpb)
No public files with extensions ('.pdf', '.lcpdf', '.txt', '.djvu', '.epub') for 'rwugbiupl1umttkibxzx4vjh4jd0c8nszblewbpb'
Found “(no title)” (ID: siytdc-Aerial_Archaeology_-_STEM_in_30_-_Season_11_-_Episode_3)
No public files with extensions ('.pdf', '.lcpdf', '.txt', '.djvu', '.epub') for 'siytdc-Aerial_Archaeology_-_STEM_in_30_-_Season_11_-_Episode_3'
Found “(no title)” (ID: swastika-remixed-third-reich-documentary)
No public files with extensions ('.pdf', '.lcpdf', '.txt', '.djvu', '.epub') for 'swastika-remixed-third-reich-documentary

voltairecollection:
 error downloading file downloads/voltairecollection/voltairecollection/VOL9.pdf, exception raised: 401 Client Error: Unauthorized for url: https://dn790000.ca.archive.org/0/items/voltairecollection/VOL9.pdf


Error downloading from voltairecollection: 401 Client Error: Unauthorized for url: https://dn790000.ca.archive.org/0/items/voltairecollection/VOL9.pdf
Found “(no title)” (ID: wcn8gzsww7tumgb4jqtbmlmidolufmmsdabftrhj)
No public files with extensions ('.pdf', '.lcpdf', '.txt', '.djvu', '.epub') for 'wcn8gzsww7tumgb4jqtbmlmidolufmmsdabftrhj'
Found “(no title)” (ID: wgcgccnc0coljdauunpmvxjptjtutyvzibqbhw8r)
No public files with extensions ('.pdf', '.lcpdf', '.txt', '.djvu', '.epub') for 'wgcgccnc0coljdauunpmvxjptjtutyvzibqbhw8r'
Found “(no title)” (ID: worldheritagecon0000cave)


worldheritagecon0000cave:
 downloading worldheritagecon0000cave.lcpdf: 100%|██████████| 20.3M/20.3M [00:14<00:00, 1.43MiB/s]
 error downloading file downloads/worldheritagecon0000cave/worldheritagecon0000cave/worldheritagecon0000cave.pdf, exception raised: 401 Client Error: Unauthorized for url: https://dn790006.ca.archive.org/0/items/worldheritagecon0000cave/worldheritagecon0000cave.pdf


Error downloading from worldheritagecon0000cave: 401 Client Error: Unauthorized for url: https://dn790006.ca.archive.org/0/items/worldheritagecon0000cave/worldheritagecon0000cave.pdf
Found “(no title)” (ID: ybpzahaflaub4f0vhelm5dt1inay8qlsshvlvso4)
No public files with extensions ('.pdf', '.lcpdf', '.txt', '.djvu', '.epub') for 'ybpzahaflaub4f0vhelm5dt1inay8qlsshvlvso4'
Found “(no title)” (ID: youtube--3oSd8DC8Kc)
No public files with extensions ('.pdf', '.lcpdf', '.txt', '.djvu', '.epub') for 'youtube--3oSd8DC8Kc'
Found “(no title)” (ID: youtube-AW55J2zE3N4)
No public files with extensions ('.pdf', '.lcpdf', '.txt', '.djvu', '.epub') for 'youtube-AW55J2zE3N4'
Found “(no title)” (ID: youtube-DmKU44ZeuRQ)
No public files with extensions ('.pdf', '.lcpdf', '.txt', '.djvu', '.epub') for 'youtube-DmKU44ZeuRQ'
Found “(no title)” (ID: youtube-FsrYR52Tv54)
No public files with extensions ('.pdf', '.lcpdf', '.txt', '.djvu', '.epub') for 'youtube-FsrYR52Tv54'
Found “(no title)” (ID: youtube-Mhs